In [26]:
# Nombre: Darlin Anacicha    Curso:GR1CC

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import time
from collections import defaultdict

#Cargamos los Datos del dataset
df = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
#df_large = pd.read_csv('/kaggle/input/rotten-tomato-movie-reviews-1-44m-rows/rotten_tomatoes_movie_reviews.csv')

df.head()
#df.head(5000)
#df_large.head(10000)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [27]:
#Parte 1: Búsqueda lineal en documentos

# Convertimos las reseñas a minúsculas una sola vez (optimización)
reviews = df['review'].astype(str).str.lower().tolist()

# También creamos un corpus pequeño para la comparación (primeras 5000 reseñas)
reviews_small = reviews[:5000]

# También creamos un corpus pequeño para la comparación (primeras 5000 reseñas)
reviews_small = reviews[:5000]

# Función para limpiar texto (quita puntuación y símbolos)
def limpiar_texto(texto):
    """Elimina signos de puntuación y deja solo letras y espacios."""
    return re.sub(r'[^a-záéíóúüñ\s]', '', texto)

def busqueda_lineal(palabra, documentos):
    palabra = palabra.lower()
    encontrados = []
    for i, texto in enumerate(documentos):
        # usamos expresiones regulares para evitar falsos positivos (ej: "cat" dentro de "catch")
        if re.search(rf'\b{re.escape(palabra)}\b', texto):
            encontrados.append(i)
    return encontrados


palabra = input("Escribe una palabra a buscar: ").strip().lower()

inicio = time.time()
resultados = busqueda_lineal(palabra, reviews)
fin = time.time()

print(f"La palabra '{palabra}' aparece en {len(resultados)} reseñas.")
print(f"Tiempo de búsqueda lineal: {round(fin - inicio, 4)} segundos")

# Mostrar algunos ejemplos
if resultados:
    print("\nPrimeras 3 reseñas donde aparece la palabra:")
    for i in resultados[:3]:
        texto_mostrado = df.loc[i, 'review'][:120].replace('\n', ' ')
        print(f"- Documento {i}: {texto_mostrado}...")
else:
    print("No se encontraron coincidencias.")

Escribe una palabra a buscar:  love


La palabra 'love' aparece en 8924 reseñas.
Tiempo de búsqueda lineal: 1.3996 segundos

Primeras 3 reseñas donde aparece la palabra:
- Documento 2: I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and...
- Documento 4: Petter Mattei's "Love in the Time of Money" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait ...
- Documento 9: If you like original gut wrenching laughter you will like this movie. If you are young or old then you will love this mo...


In [28]:
#Parte 2: Construcción de un índice invertido

def construir_indice(documentos):
    indice = defaultdict(list)
    for i, texto in enumerate(documentos):
        # elimina signos de puntuación
        texto_limpio = limpiar_texto(texto)
        for palabra in set(texto_limpio.split()):
            indice[palabra].append(i)
    return indice
    
inicio = time.time()
indice_invertido = construir_indice(reviews)
fin = time.time()

print(f"\nIndice invertido creado con éxito en {round(fin - inicio, 2)} segundos.")
print(f"Total de palabras indexadas: {len(indice_invertido)}")


Indice invertido creado con éxito en 9.03 segundos.
Total de palabras indexadas: 176035


In [29]:
#Parte 3: Evaluación de tiempos de búsqueda

palabra = input("\nPalabra para comparar tiempos: ").strip().lower()

# --- Corpus pequeño ---
t1 = time.time()
res_lineal_peq = busqueda_lineal(palabra, reviews_small)
t2 = time.time()

t3 = time.time()
res_indice_peq = [i for i in indice_invertido.get(palabra, []) if i < 5000]
t4 = time.time()

# --- Corpus grande ---
t5 = time.time()
res_lineal_grande = busqueda_lineal(palabra, reviews)
t6 = time.time()

t7 = time.time()
res_indice_grande = indice_invertido.get(palabra, [])
t8 = time.time()



Palabra para comparar tiempos:  love


In [30]:
datos = {
    "Método": ["Búsqueda lineal", "Índice invertido"],
    "Corpus pequeño (s)": [round(t2 - t1, 4), round(t4 - t3, 6)],
    "Corpus grande (s)": [round(t6 - t5, 4), round(t8 - t7, 6)],
    "Resultados (grande)": [len(res_lineal_grande), len(res_indice_grande)]
}

tabla = pd.DataFrame(datos)
print("\nTabla comparativa de tiempos:")
print(tabla.to_string(index=False))


Tabla comparativa de tiempos:
          Método  Corpus pequeño (s)  Corpus grande (s)  Resultados (grande)
 Búsqueda lineal            0.133600           1.288300                 8924
Índice invertido            0.001271           0.000108                 8696


In [31]:
print("\nConclusión:")
print("La búsqueda con índice invertido es significativamente más rápida,")
print("especialmente en corpus grandes, porque evita recorrer todos los documentos.")



Conclusión:
La búsqueda con índice invertido es significativamente más rápida,
especialmente en corpus grandes, porque evita recorrer todos los documentos.
